## Import Modules

In [2]:
import requests
import pandas as pd
from tqdm import tqdm 
import pickle
#import esm
import torch

## Data Preprocess

In [4]:
train = pd.read_csv("/data/home/yjkim/DACON/Cancer/train.csv")
test = pd.read_csv("/data/home/yjkim/DACON/Cancer/test.csv")

In [5]:
prot_list = list(train.drop(columns=['SUBCLASS', 'ID']).columns)

In [4]:
#받아둔 protein sequence dictionary 열기
with open("./protein_sequence_database/gene_seq_dict.pkl", "rb") as f:
    prot_seq_dict = pickle.load(f)

In [6]:
X_train = train.drop(columns=['ID', 'SUBCLASS'])

In [10]:
length_error = []
position_error = []
no_seq_error = []

In [11]:
import re

def split_string(s):
    # 정규 표현식으로 영문자-숫자-영문자를 그룹화
    match = re.match(r'([a-zA-Z]+)(\d+)([a-zA-Z]+)', s)
    
    if match:
        # 그룹화된 결과를 튜플로 반환
        return match.groups()
    else:
        return None
    
    
def get_wt_seq(gene_symbol, prot_seq_dict):
    '''
    gene_symbol : 유전자 이름 (ex. ABCC2)
    prot_seq_dict : {Protein : Amino Acid Sequence}
    '''
    
    return prot_seq_dict[gene_symbol]

def get_mut_seq(gene_symbol, sequence, mutations):
    '''
    sequence : 원래 sequence
    mutations : 변이 string. 여러 변이가 white character로 붙어있는 경우도 존재 (ex. 'C166Y', 'R3391Q R2777I L2648I E1706D R1324C' , ...])
    '''
    try:
        sequence = list(sequence)
    except TypeError:
        print("No sequence found")
        no_seq_error.append([gene_symbol, mutations])
        return None
    mutations = mutations.split()
    # print(len(mutations))
    for mutation in mutations:
        if 'del' in mutation:
            match = re.match(r'([a-zA-Z]*)(\d+)del', mutation)
            pos = int(match.groups()[1]) - 1
            if pos < len(sequence):
                sequence = sequence[:pos] + sequence[pos+1:]
            else:
                # print(f"lenght error: Expected >= {pos + 1}, got {len(sequence)}")
                length_error.append([gene_symbol, mutation])
            
        elif '>' in mutation:  # 삽입/삭제 변이
            original, new = mutation.split('>')
            start = int(original.split('_')[0]) - 1
            end = int(original.split('_')[1][:-2]) if '_' in original else start + 1
            if '*' in new:
                if new[0] == "*":
                    sequence = sequence[:start]
                elif new[1] == "*":
                    sequence = sequence[:start] + list(new[1])
            else:
                sequence = sequence[:start] + list(new) + sequence[end:]
        
        elif mutation[-1] == '*':  # 종결 돌연변이
            # print(change[1:-1])
            pos = int(mutation[1:-1]) - 1
            if pos < len(sequence):
                if sequence[pos] == mutation[0]:
                    sequence = sequence[:pos]
                else:
                    # print(f"position error at position {pos + 1}: {sequence[pos]} instead of {mutation[0]}")
                    position_error.append([gene_symbol, mutation])
                    # raise ValueError(f"Unexpected amino acid at position {pos + 1}: {sequence[pos]} instead of {original}")
            else:
                # print(f"lenght error: Expected >= {pos + 1}, got {len(sequence)}")
                length_error.append([gene_symbol, mutation])
                # raise ValueError(f"Peptide is shorter than the length: Expected >= {pos + 1}, got {len(sequence)}")

            break
        elif 'fs' in mutation:  # 프레임 시프트 돌연변이
            if '-' in mutation:
                pos = int(mutation[1:-2]) - 1
                if pos < len(sequence):
                    sequence = sequence[:pos + 1]
                else:
                    # print(f"lenght error: Expected >= {pos + 1}, got {len(sequence)}")
                    length_error.append([gene_symbol, mutation])
            elif '*' in mutation:
                index = mutation.index('*')
                pos = int(mutation[index+1:-2]) - 1
                if pos < len(sequence):
                    sequence = sequence[:pos + 1]
                else:
                    # print(f"lenght error: Expected >= {pos + 1}, got {len(sequence)}")
                    length_error.append([gene_symbol, mutation])

            else:
                # pos = int(mutation[1:-2]) - 1
                pos = int(split_string(mutation)[1]) - 1
                if pos < len(sequence):
                    if sequence[pos] == split_string(mutation)[0]:
                        sequence = sequence[:pos + 1]  # 이후의 서열을 무시
                    else:
                        # print(f"position error at position {pos + 1}: {sequence[pos]} instead of {mutation[0]}")
                        position_error.append([gene_symbol, mutation])
                else:
                    # print(f"lenght error: Expected >= {pos + 1}, got {len(sequence)}")
                    length_error.append([gene_symbol, mutation])
                    # raise ValueError(f"Peptide is shorter than the length: Expected >= {pos + 1}, got {len(sequence)}")
                break
                
        else:  # 점 돌연변이
            # print(change)
            original = mutation[0]
            # print(change[1:-1])
            pos = int(mutation[1:-1]) - 1
            # print(pos)
            if pos < len(sequence):
                new = mutation[-1]
                if sequence[pos] == original:
                    sequence[pos] = new
                else:
                    # print(f"position error at position {pos + 1}: Sequence:{sequence[pos]}, Expected:{original}")
                    position_error.append([gene_symbol, mutation])
                    # raise ValueError(f"Unexpected amino acid at position {pos + 1}: {sequence[pos]} instead of {original}")
            else:
                # print(f"lenght error: Expected >= {pos + 1}, got {len(sequence)}")
                length_error.append([gene_symbol, mutation])
                # raise ValueError(f"Peptide is shorter than the length: Expected >= {pos + 1}, got {len(sequence)}")

    return ''.join(sequence)
    

In [ ]:
# Example: for sample train_0005, get original and mutated sequence of ABCC2
# C166Y

ABCC2_seq = get_wt_seq('ABCC2', prot_seq_dict)
ABCC2_mutations = X_train.loc[5]['ABCC2']
mutated_seq = get_mut_seq('ABCC2', ABCC2_seq, ABCC2_mutations)
print(ABCC2_seq[165])
print(ABCC2_mutations)
print(mutated_seq[165])

In [17]:
# Example: for sample train_0047, get original and mutated sequence of COL6A3 
# R1881C R97C

COL6A3_seq = get_wt_seq('COL6A3', prot_seq_dict)
COL6A3_mutations = X_train.loc[47]['COL6A3']
print(COL6A3_mutations)
mutated_seq = get_mut_seq('COL6A3', COL6A3_seq, COL6A3_mutations)
print(COL6A3_seq[1880], COL6A3_seq[96]) 
print(mutated_seq[1880], mutated_seq[96])

R1881C R97C
R R
C C


In [18]:
# Example: for sample train_1239, get original and mutated sequence of FBXW7 
# E113*

seq = get_wt_seq('FBXW7', prot_seq_dict)
print(len(seq))
mut = X_train.loc[1239]['FBXW7']
print(mut)
mutated_seq = get_mut_seq('FBXW7', seq, mut)
print(seq)
print("====")
print(mutated_seq)

85
E113*
MTMFKGSNEMKSRWNWGSITCIICFTCVGSQLSMSSSKASNFSGPLQLYQRELEIFIVLTDVPNYRLIKENSHLHTTIVDQGRTV
====
MTMFKGSNEMKSRWNWGSITCIICFTCVGSQLSMSSSKASNFSGPLQLYQRELEIFIVLTDVPNYRLIKENSHLHTTIVDQGRTV


In [19]:
length_error

[['FBXW7', 'E113*']]

### uniprot protein seq retrieval 예시

In [20]:
def get_uniprot_id_(gene_symbol):
    # Homo sapiens 필터 추가
    url = f"https://rest.uniprot.org/uniprotkb/search?query=gene:{gene_symbol}+AND+organism_id:9606&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            uniprot_id = data['results'][0]['primaryAccession']
            return uniprot_id
        else:
            return None
    else:
        response.raise_for_status()

def get_uniprot_id(entry_name):
    # entry name을 직접 검색
    url = f"https://rest.uniprot.org/uniprotkb/search?query={entry_name}+AND+organism_id:9606&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            uniprot_id = data['results'][0]['primaryAccession']
            return uniprot_id
        else:
            return None
    else:
        response.raise_for_status()

# 예시 사용
entry_name = "FBXW7_HUMAN"
uniprot_id = get_uniprot_id(entry_name)
print(uniprot_id)


entry_name = "FBXW7"
uniprot_id = get_uniprot_id_(entry_name)
print(uniprot_id)


Q969H0
B0L3A2


In [102]:
mutated_seq = get_mut_seq(ABCC2_seq, mutations)

C166Y
C166Y
166


In [104]:
mutated_seq[165]

'Y'

## Let's get the list of wrong protein sequences.

In [20]:
mut_list = []

for row in tqdm(X_train.index):
    for col in X_train.columns:
        value = X_train.at[row, col]
        if value != 'WT':
            mut_list.append([row, col, value])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6201/6201 [02:07<00:00, 48.67it/s]


In [21]:
len(mut_list)

218893

In [22]:
mut_list[:5] # sample number, gene, mutation

[[0, 'ABCC2', 'D623D'],
 [0, 'ADCY2', 'F736C'],
 [0, 'CD86', 'P117R'],
 [0, 'EPHB4', 'R305S'],
 [0, 'FOXC2', 'N23K']]

In [25]:
length_error = []
position_error = []
no_seq_error = []
for mut in tqdm(mut_list):
    seq = get_wt_seq(mut[1], prot_seq_dict)
    mutated_seq = get_mut_seq(mut[1], seq, mut[2])

  5%|███████▏                                                                                                                                                   | 10171/218893 [00:00<00:04, 50382.36it/s]

No sequence found
No sequence found
No sequence found
No sequence found
No sequence found
No sequence found
No sequence found
No sequence found


  9%|██████████████▌                                                                                                                                            | 20622/218893 [00:00<00:03, 51743.42it/s]

No sequence found
No sequence found
No sequence found
No sequence found
No sequence found


 16%|█████████████████████████▌                                                                                                                                 | 36018/218893 [00:00<00:03, 49686.93it/s]

...

No sequence found
No sequence found
No sequence found


 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 210860/218893 [00:04<00:00, 49739.77it/s]

No sequence found
No sequence found
No sequence found
No sequence found


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 218893/218893 [00:04<00:00, 50728.72it/s]

No sequence found
No sequence found
No sequence found


In [23]:
len(position_error)

12225

In [24]:
len(length_error)

1221

In [28]:
genes_position_err = [error[0] for error in position_error]
genes_length_err = [error[0] for error in length_error]
genes_no_seq = [error[0] for error in no_seq_error]
error_genes = set(genes_position_err + genes_length_err + genes_no_seq)

In [29]:
len(error_genes)

2215

In [27]:
error_genes

{'PES1',
 'CDKN3',
 'REL',
 'ERCC4',
 'SDC4',
 'RAD23B',
 'PCOLCE',
 'ATXN1',
 'HK1',
 'ARRB2',
 'TNNT2',
 'NR4A3',
 'SH2B2',
 'C2',
 'FUT4',
 'NDC80',
 'EDIL3',
 'KIF5C',
 'CS',
 'GLMN',
 'PSPH',
 'CENPJ',
 'DAB2',
 'PDLIM5',
 'NELFB',
 'HLA-DRB1',
 'TNFRSF9',
 'UBE2D3',
 'BDNF',
 'CYP2E1',
 'TPST2',
 'VHL',
 'GCA',
 'DPP4',
 'TNNI1',
 'RPS6KA2',
 'CRMP1',
 'TROAP',
 'ERN1',
 'IL2',
 'PML',
 'CMBL',
 'CRLF1',
 'CD86',
 'PROC',
 'IL1RL2',
 'RETSAT',
 'CDKAL1',
 'IPCEF1',
 'LUC7L3',
 'PHF7',
 'ING3',
 'AK1',
 'GZMA',
 'SPI1',
 'PHLDA1',
 'ABCB8',
 'SYK',
 'SERPINE1',
 'SF3A1',
 'SPC25',
 'EFNA5',
 'SYTL2',
 'MAP3K1',
 'P4HA1',
 'VEZF1',
 'CUL1',
 'CHRNA5',
 'RTN3',
 'GPR19',
 'CASP2',
 'CLDN15',
 'NIPBL',
 'ELANE',
 'HDDC2',
 'TEAD4',
 'MX2',
 'AUTS2',
 'PKD2',
 'PDPN',
 'G6PC2',
 'NDUFC2',
 'NFKB2',
 'EPB41L3',
 'NKIRAS1',
 'BAG1',
 'VAV3',
 'PRDX3',
 'CDC25A',
 'GPC1',
 'HMGCL',
 'ZNF112',
 'EIF2S3',
 'IL10RA',
 'EXOSC7',
 'LPIN1',
 'PEX26',
 'COG2',
 'COL1A2',
 'COPB1',
 'IL18RAP',
 

In [118]:
for _, _, muts in mut_list:
    for mut in muts.split():
        if '*' in mut:
            if 'fs' in mut:
                print(mut)

IW*44fs
*214fs
*444fs
